<a href="https://www.kaggle.com/sdysch/bacteria-clf-randomforest-with-hyperopt-tuning?scriptVersionId=88752384" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [41]:
# found this competition ~2 days before the submission deadline - trying a *really* quick attempt :/
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [42]:
df_train = pd.read_csv('../input/tabular-playground-series-feb-2022/train.csv')
df_test  = pd.read_csv('../input/tabular-playground-series-feb-2022/test.csv')

In [43]:
print(df_train.columns)
print(df_test.columns)

Index(['row_id', 'A0T0G0C10', 'A0T0G1C9', 'A0T0G2C8', 'A0T0G3C7', 'A0T0G4C6',
       'A0T0G5C5', 'A0T0G6C4', 'A0T0G7C3', 'A0T0G8C2',
       ...
       'A8T0G1C1', 'A8T0G2C0', 'A8T1G0C1', 'A8T1G1C0', 'A8T2G0C0', 'A9T0G0C1',
       'A9T0G1C0', 'A9T1G0C0', 'A10T0G0C0', 'target'],
      dtype='object', length=288)
Index(['row_id', 'A0T0G0C10', 'A0T0G1C9', 'A0T0G2C8', 'A0T0G3C7', 'A0T0G4C6',
       'A0T0G5C5', 'A0T0G6C4', 'A0T0G7C3', 'A0T0G8C2',
       ...
       'A8T0G0C2', 'A8T0G1C1', 'A8T0G2C0', 'A8T1G0C1', 'A8T1G1C0', 'A8T2G0C0',
       'A9T0G0C1', 'A9T0G1C0', 'A9T1G0C0', 'A10T0G0C0'],
      dtype='object', length=287)


In [44]:
df_train.drop(['row_id'], axis=1, inplace=True)
df_test.drop(['row_id'], axis=1, inplace=True)
print(df_train.columns)
print(df_test.columns)

Index(['A0T0G0C10', 'A0T0G1C9', 'A0T0G2C8', 'A0T0G3C7', 'A0T0G4C6', 'A0T0G5C5',
       'A0T0G6C4', 'A0T0G7C3', 'A0T0G8C2', 'A0T0G9C1',
       ...
       'A8T0G1C1', 'A8T0G2C0', 'A8T1G0C1', 'A8T1G1C0', 'A8T2G0C0', 'A9T0G0C1',
       'A9T0G1C0', 'A9T1G0C0', 'A10T0G0C0', 'target'],
      dtype='object', length=287)
Index(['A0T0G0C10', 'A0T0G1C9', 'A0T0G2C8', 'A0T0G3C7', 'A0T0G4C6', 'A0T0G5C5',
       'A0T0G6C4', 'A0T0G7C3', 'A0T0G8C2', 'A0T0G9C1',
       ...
       'A8T0G0C2', 'A8T0G1C1', 'A8T0G2C0', 'A8T1G0C1', 'A8T1G1C0', 'A8T2G0C0',
       'A9T0G0C1', 'A9T0G1C0', 'A9T1G0C0', 'A10T0G0C0'],
      dtype='object', length=286)


In [45]:
# clean duplicates
print(df_train.duplicated().sum())
df_train = df_train.drop_duplicates()
print(df_train.duplicated().sum())

76007
0


In [46]:
print(df_train.isna().sum().sum())
print(df_test.isna().sum().sum())

0
0


In [47]:
print(df_train['target'])

0           Streptococcus_pyogenes
1              Salmonella_enterica
2              Salmonella_enterica
3              Salmonella_enterica
4               Enterococcus_hirae
                    ...           
199989       Klebsiella_pneumoniae
199990            Escherichia_coli
199991        Bacteroides_fragilis
199995         Salmonella_enterica
199997    Streptococcus_pneumoniae
Name: target, Length: 123993, dtype: object


In [48]:
# class imbalance
n_samples = df_train['target'].value_counts().sort_index()
n_samples /= n_samples.sum().sum()
print(n_samples)

Bacteroides_fragilis        0.100990
Campylobacter_jejuni        0.100562
Enterococcus_hirae          0.099788
Escherichia_coli            0.099175
Escherichia_fergusonii      0.099078
Klebsiella_pneumoniae       0.100167
Salmonella_enterica         0.099925
Staphylococcus_aureus       0.100127
Streptococcus_pneumoniae    0.100135
Streptococcus_pyogenes      0.100054
Name: target, dtype: float64


In [49]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
df_train['target'] = enc.fit_transform(df_train['target'])
print(df_train['target'])

0         9
1         6
2         6
3         6
4         2
         ..
199989    5
199990    3
199991    0
199995    6
199997    8
Name: target, Length: 123993, dtype: int64


In [50]:
# split training into train + validation for model testing
from sklearn.model_selection import train_test_split

X = df_train.drop(['target'], axis=1)
y = df_train['target']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [52]:
# check class imbalance in train/val splits
# class imbalance
n_samples = y_train.value_counts().sort_index()
n_samples /= n_samples.sum().sum()
print(n_samples)

# check class imbalance in train/val splits
# class imbalance
n_samples = y_val.value_counts().sort_index()
n_samples /= n_samples.sum().sum()
print(n_samples)

0    0.100853
1    0.100823
2    0.099512
3    0.099089
4    0.099149
5    0.100208
6    0.099895
7    0.100268
8    0.100117
9    0.100087
Name: target, dtype: float64
0    0.101536
1    0.099520
2    0.100891
3    0.099520
4    0.098794
5    0.100004
6    0.100044
7    0.099560
8    0.100206
9    0.099923
Name: target, dtype: float64


# hyperparameter tuning

In [74]:
# defining search space
from hyperopt import hp
mln = [v + 1 for v in range(1, 20)]
mln.append(None)
print(mln)

params = {
    'n_estimators'   : hp.quniform('n_estimators', 50, 200, 25),
    'max_depth'      : hp.quniform('max_depth', 1, 20, 1),
    'max_leaf_nodes' : hp.choice('max_leaf_nodes', mln),
    'max_features'   : hp.choice('max_features', ['sqrt', 'log2']),
}

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, None]


# Objective function

In [75]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from hyperopt import STATUS_OK

def objective(params):
    global X_train, y_train, X_val, y_val
    
    params['n_estimators']   = int(params['n_estimators'])
    params['max_depth']      = int(params['max_depth'])
    
    # fit model
    model = RandomForestClassifier(**params, random_state=42)
    model.fit(X_train, y_train)
    
    # make predictions with fitted model
    y_pred = model.predict(X_val)
    
    # metrics
    accuracy  = accuracy_score(y_val, y_pred)
    # return metrics
    return {
        'loss'     : -accuracy,
        'status'   : STATUS_OK,
        'accuracy' : accuracy,
        }

# run trials

In [ ]:
from hyperopt import fmin, tpe, Trials

trials = Trials()
best = fmin(objective,
            space = params,
            algo = tpe.suggest,
            max_evals = 50,
            trials = trials,
           max_queue_len = 5)
print(best)

 30%|███       | 15/50 [07:43<19:31, 33.48s/trial, best loss: -0.969756845034074] 